In [18]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [19]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-no7mq8rs
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-no7mq8rs
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [20]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [21]:
%%cu
#include <iostream>

using namespace std;


// Kernel function to perform matrix multiplication
__global__ void multiply(int* A, int* B, int* C, int size) {
    // Uses thread indices and block indices to compute each element

    // Calculate the row and column indices of the current thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Ensure current thread is within the matrix bounds
    if (row < size && col < size) {
        int sum = 0;
        // Perform the dot product of the row of A and column of B
        for (int i = 0; i < size; i++) {
            sum += A[row * size + i] * B[i * size + col];
        }
        // Store the result in the corresponding element of matrix C
        C[row * size + col] = sum;
    }
}

// Function to initialize a matrix with random values
void initialize(int* matrix, int size) {
    for (int i = 0; i < size * size; i++) {
        matrix[i] = rand() % 10;  // Generate random values between 0 and 9
    }
}

// Function to print a matrix
void print(int* matrix, int size) {
    for (int row = 0; row < size; row++) {
        for (int col = 0; col < size; col++) {
            cout << matrix[row * size + col] << " ";
        }
        cout << '\n';
    }
    cout << '\n';
}

int main() {
    // Declare pointers for matrices A, B, and C
    int* A, * B, * C;

    int N = 2;              // Size of matrices
    int blockSize = 16;     // Block size for CUDA threads

    int matrixSize = N * N;  // Total number of elements in matrices
    size_t matrixBytes = matrixSize * sizeof(int);  // Total memory size for matrices

    // Allocate memory for matrices A, B, and C on the host (CPU)
    A = new int[matrixSize];
    B = new int[matrixSize];
    C = new int[matrixSize];

    // Initialize matrices A and B with random values
    initialize(A, N);
    initialize(B, N);

    cout << "Matrix A: \n";
    print(A, N);

    cout << "Matrix B: \n";
    print(B, N);

    int* X, * Y, * Z;

    // Allocate space for matrices X, Y, and Z on the device (GPU)
    cudaMalloc(&X, matrixBytes);
    cudaMalloc(&Y, matrixBytes);
    cudaMalloc(&Z, matrixBytes);

    // Copy values from matrices A and B on host to matrices X and Y on device
    cudaMemcpy(X, A, matrixBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(Y, B, matrixBytes, cudaMemcpyHostToDevice);

    // Define grid and block dimensions for CUDA kernel execution
    dim3 grid(N, N);  // 2D grid with N x N blocks
    dim3 block(blockSize, blockSize);  // 2D block with blockSize x blockSize threads

    // Launch the kernel function for matrix multiplication
    multiply<<<grid, block>>>(X, Y, Z, N);

    // Copy the result matrix Z from the device to the host
    cudaMemcpy(C, Z, matrixBytes, cudaMemcpyDeviceToHost);

    cout << "Multiplication of matrix A and B: \n";
    print(C, N);

    // Free memory on the host and device
    delete[] A;
    delete[] B;
    delete[] C;
    cudaFree(X);
    cudaFree(Y);
    cudaFree(Z);

    return 0;
}


Matrix A: 
3 6 
7 5 

Matrix B: 
3 5 
6 2 

Multiplication of matrix A and B: 
45 27 
51 45 


